In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from final_project.Trading_Project.utils.errors_handler import error_tracking_decorator, get_root_cause_error

def plus(a, b):
    return a + b


@error_tracking_decorator
def my_func():
    result = plus("1",2)

r, s = my_func()
s

'Error in function my_func \n===================================================\nFilename: /tmp/ipykernel_79758/746703035.py,\nFunction name: my_func, params: {}\n\t-----\n\tLine: 9,     result = plus("1",2)\n \t-----\n\tNote: This error is from your code\n===================================================\nFilename: /tmp/ipykernel_79758/746703035.py,\nFunction name: plus, params: {\'a\': \'1\', \'b\': 2}\n\t-----\n\tLine: 4,     return a + b\n \t-->ROOT CAUSE: TypeError: can only concatenate str (not "int") to str \n\t-----\n\tNote: This error is from your code\n'

In [17]:
ss = [('\n'.join([ssi for ssi in s.split('\n') if '-----' not in ssi and 'Note' not in ssi and 'Filename' not in ssi])).strip().replace('\t', '') for s in s.split("===================================================")]
ss


['Error in function my_func',
 'Function name: my_func, params: {}\nLine: 9,     result = plus("1",2)',
 'Function name: plus, params: {\'a\': \'1\', \'b\': 2}\nLine: 4,     return a + b\n -->ROOT CAUSE: TypeError: can only concatenate str (not "int") to str']

In [54]:

processed_strings = [process_string(clean_string(string)) for string in s.split("===================================================")]
processed_strings


['Error in function my_func',
 "Function name: my_func, params: {}\nLine: 9,     result = plus('1',2)",
 'Function name: plus, params: {\'a\': \'1\', \'b\': 2}\nLine: 4,     return a + b\n -->ROOT CAUSE: TypeError: can only concatenate str (not "int") to str ']

In [9]:
from final_project.Trading_Project.utils.file_text_handler import get_code_from_text
import backtrader as bt
from final_project.Trading_Project.base_strategy import BaseStrategy


# exec(get_code_from_text("```python\nimport backtrader as bt\n\nclass CustomIchimokuIndicator(bt.Indicator):\n    \"\"\"\n    Custom Ichimoku Cloud indicator.\n    \"\"\"\n    lines = (\"tenkan\", \"kijun\", \"senkou_a\", \"senkou_b\")\n    params = ((\"tenkan_period\", 9), (\"kijun_period\", 26), (\"senkou_period\", 52))\n\n    def __init__(self):\n        self.addminperiod(self.params.tenkan_period)\n        self.addminperiod(self.params.kijun_period)\n        self.addminperiod(self.params.senkou_period)\n\n    def next(self):\n        high = self.data.high\n        low = self.data.low\n        close = self.data.close\n\n        tenkan = (high.rolling(window=self.params.tenkan_period).max() + low.rolling(window=self.params.tenkan_period).min()) / 2\n        kijun = (high.rolling(window=self.params.kijun_period).max() + low.rolling(window=self.params.kijun_period).min()) / 2\n        senkou_a = (tenkan + kijun) / 2\n        senkou_b = (high.rolling(window=self.params.senkou_period).max() + low.rolling(window=self.params.senkou_period).min()) / 2\n\n        self.lines.tenkan[0] = tenkan[0]\n        self.lines.kijun[0] = kijun[0]\n        self.lines.senkou_a[0] = senkou_a[0]\n        self.lines.senkou_b[0] = senkou_b[0]\n\n\nclass BackTestStrategy(BaseStrategy):\n    def __init__(self, *args, **kwargs):\n        super().__init__(*args, **kwargs)\n        self.ichimoku = CustomIchimokuIndicator()\n\n    def execute(self):\n        \"\"\"\n        Define the trading logic based on the Ichimoku Cloud indicator.\n\n        Returns:\n        int: Trading signal: 1 (long), -1 (sell), or None if no signal.\n        \"\"\"\n        if self.ichimoku.tenkan[0] > self.ichimoku.kijun[0] and self.ichimoku.senkou_a[0] > self.ichimoku.senkou_b[0]:\n            return 1  # Long signal\n        elif self.ichimoku.tenkan[0] < self.ichimoku.kijun[0] and self.ichimoku.senkou_a[0] < self.ichimoku.senkou_b[0]:\n            return -1  # Short signal\n        return None  # No signal\n```"), globals())


import math

class GannAngle(bt.Indicator):
    lines = ("angle",)
    params = (("period", 1),)

    def __init__(self):
        self.addminperiod(self.params.period)

    def next(self):
        if len(self) == 1:
            self.lines.angle[0] = 0  # Initialize Gann Angle at the beginning
        else:
            high = self.data.high[0]
            low = self.data.low[0]
            close = self.data.close[0]
            self.lines.angle[0] = math.atan((high - low) / close) * (180 / math.pi)

class BackTestStrategy(BaseStrategy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Initialize the Gann Angle indicator
        self.gann_angle = GannAngle()
        # Initialize the RSI indicator
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=14)

    def execute(self):
        """
        Define the trading logic based on the Gann Angle and RSI indicators.

        Returns:
        int: Trading signal: 1 (long), -1 (sell), or None if no signal.
        """
        if self.gann_angle[0] > 45 and self.rsi[0] > 50:  # Buy condition: Gann Angle indicates support and RSI is above 50
            return 1  # Long signal
        elif self.gann_angle[0] < 45 and self.rsi[0] < 50:  # Sell condition: Gann Angle indicates resistance and RSI is below 50
            return -1  # Short signal
        return None  # No signal

In [6]:
from final_project.Trading_Project.data_loader import load_stock_data

data = [bt.feeds.PandasData(
                dataname=load_stock_data(ticker='AAPL', period="1y"), datetime="Date", 
                timeframe=bt.TimeFrame.Minutes)]



[*********************100%%**********************]  1 of 1 completed


In [10]:
from final_project.Trading_Project.my_strategy import BackTestStrategy

class MyBuySell(bt.observers.BuySell):
    plotlines = dict(
        buy=dict(marker='$\u21E7$', markersize=12.0),
        sell=dict(marker='$\u21E9$', markersize=12.0)
    )

class CelebroCreator:
    def __init__(self, strategy, list_of_data, stake=100, cash=20000):
        self.cerebro = bt.Cerebro(cheat_on_open=True)

        self.cerebro.addstrategy(strategy)
        self.cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
        self.cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')
        self.cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
        self.cerebro.addobserver(bt.observers.Value)

        for data in list_of_data:
            self.cerebro.adddata(data)
        self.cerebro.broker.set_cash(cash)
        bt.observers.BuySell = MyBuySell
        
        # PREVIOUS
        # self.strats = self.cerebro.run()
        
        # CURRENT
        self.strats, self.message = self._run_cerebro()

    
    # CURRENT
    @error_tracking_decorator
    def _run_cerebro(self):
        self.cerebro.run()


    def show(self):
        print("Final Portfolio Value: %.2f" % self.cerebro.broker.getvalue())
        print("Total point return: ", (self.cerebro.broker.getvalue() - self.cerebro.broker.startingcash))

        try:
            sharpe_ratio = self.strats[0].analyzers.SharpeRatio.get_analysis()['sharperatio']
        except:
            print('No Buy/Sell Signal! No Sharpe Ratio!')
        # Plot the results
        figs = self.cerebro.plot(
            iplot=False, 
            # style="pincandle", 
            # width=60 * 10, height=40 * 10,
            figsize=(100, 80),
            # sharpe_ratio=sharpe_ratio
        )

# def check_valid_code(strategy, list_data):
    
#     @error_tracking_decorator
#     def extract_error_message(strategy, list_data):
#         CelebroCreator(strategy,list_data)
        
#     message = extract_error_message(BackTestStrategy, data)
#     root_error = get_root_cause_error(message) if message else ""
#     return root_error
def check_valid_code(strategy, list_data):
    
    obj = CelebroCreator(strategy,list_data)
        
    message = obj.message
    print(message)
    root_error = get_root_cause_error(message) if message else ""
    return root_error

error = check_valid_code(BackTestStrategy, data)

Error in function _run_cerebro 
Filename: /tmp/ipykernel_79758/2660629132.py,
Function name: _run_cerebro, params: {}
	-----
	Line: 34,         self.cerebro.run()
 	-----
	Note: This error is from your code
Filename: /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/backtrader/cerebro.py,
Function name: run, params: {'kwargs': {}, 'pkeys': "odict_keys(['preload', 'runonce'...", 'iterstrats': '<itertools.product object at 0x7...', 'iterstrat': "((<class 'final_project.Trading_..."}
	-----
	Line: 1132,                 runstrat = self.runstrategies(iterstrat)
 	-----
	Note: This error is from site-packages
Filename: /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/backtrader/cerebro.py,
Function name: runstrategies, params: {'iterstrat': "((<class 'final_project.Trading_...", 'predata': False, 'runstrats': '[<final_project.Trading_Project....', 'data': '<backtrader.feeds.pandafeed.Pand...', 'stratcls': "<class 'final_project.Trading_Pr...", 'sargs': None

In [12]:
error[-1]

'Line: 21,         tenkan = (high.rolling(window=self.params.tenkan_period).max() + low.rolling(window=self.params.tenkan_period).min()) / 2'